# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2017, 9, 29, 19, 22, 19, 512678, tzinfo=datetime.timezone.utc), bid=1.1825, bidSize=18169000, ask=1.18255, askSize=1000000, prevBid=1.18255, prevBidSize=17169000, prevAsk=1.1826, prevAskSize=2000000, high=1.18325, low=1.17725, close=1.1787)

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.182525

The following cell will start a 60 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.setCallback('pendingTickers', onPendingTickers)
ib.sleep(60)

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,11169000,1.1824,1.18245,3169000,1.18325,1.17725,1.1787
1,USDJPY,8000000,112.455,112.46,4000000,112.74,112.215,112.35
2,GBPUSD,1000000,1.34155,1.3416,3649000,1.34435,1.335,1.3442
3,USDCHF,2200000,0.96795,0.968,2200000,0.9721,0.967,0.9701
4,USDCAD,10700000,1.24675,1.24685,5200000,1.2533,1.2417,1.2427
5,AUDUSD,2255000,0.78455,0.7846,2000000,0.7856,0.78165,0.78575


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

In [8]:
ib.disconnect()